# Preparación, Embeddings

## Tokenize text

El primer paso será configurar una función que convierta el texto de entrada a tokens individuales, como paso de preprocesamiento para crear embeddings.

Se utiliza el text The Veredict que ha sido cedidio para poder entrenar LLM. https://en.wikisource.org/wiki/The_Verdict

In [5]:
with open("the-veredict.txt", 'r', encoding='utf-8') as f:
    raw_text = f.read()

print("The text has", len(raw_text), "characters")
print(raw_text[:99])

The text has 20480 characters
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Para separar las palabras del texto en pos de tokenizarlas podemos utilizar re.split para separar las palabras en función de los espacios en blanco que dejan. En este caso se están separando las palabras en algunos casos con signos de puntuación, lo que querremos corregir más adelante. En este caso, a diferencia de otras pruebas que hemos realizado, se mantiene la capitalización de las palabras porque sirve para que el modelo pueda distinguir entre nombres propios y comunes, entender la estructura de la oración y para que genere texto correctamente capitalizado.

In [6]:
import re
text = 'Hello, world. This, is a test.'
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


El siguiente código permite separar también los signos de puntuación, y en la celda a continuación eliminaremos los espacios en blanco. Esta decisión va a depender del tipo de modelo que se esté entrenando y de nuestros recursos disponibles, ya que aunque reducen los requerimientos, alugnos modelos son muy sensibles a la estructura del texto y a los espacios e indentaciones.

In [7]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [8]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


Añadimos más tipos de puntuación que puedan aparecer en el texto.

In [9]:
text = 'Hello, world. This, is a test?. -- This is the end.'
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '?', '.', '--', 'This', 'is', 'the', 'end', '.']


Una vez definido el tokenizador básico se puede aplicar al texto "The Veredict".

In [10]:
def tokenizador(text): 
    result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
    return [item for item in result if item.strip()]

preprocessed = tokenizador(raw_text)
print(len(preprocessed))

4690


In [11]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## Tokens to ID

Para poder mapear los tokens a IDs hay que construir un vocabulario primero, que define como se mapea cada palabra y carácter único a un número entero.

En el ejemplo que estamos trabajando construimos una lista de todos los tokens únicos y los ordenamos alfabéticamente para determinar el tamaño del vocabulario.

In [12]:
words = sorted(list(set(preprocessed)))
vocab_size = len(words)
print(vocab_size)

vocab = {token: i for i, token in enumerate(words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break

1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


Con este diccionario podemos convertir el texto en IDs y, posteriormente invertir la versión del vocabuilario para reconvertir de ID a texto.

Creamos una clase que incluya el tokenizador, el encoder y el decoder para poder palicarlo al texto.

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def tokenizador(self, text):
        result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        return [item for item in result if item.strip()]

    def encode(self, text):
        preprocessed = tokenizador(text)
        ids = [self.str_to_int[word] for word in preprocessed]
        return ids

    def decode (self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?_!"()\']|--)', r'\1', text)
        return text

In [14]:
tokenizer = SimpleTokenizerV1(vocab)

Probamos que el encoder funciona correctamente

In [15]:
text = """"It's the last he painted, you know," Mrs. Gisburn said"""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851]


Probamos ahora que podemos recuperar el texto tras codificarlo

In [17]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said


El problema sucede cuando tratamos de aplicar el tokenizador con palabras que no figuran en el texto de entrenamiento, por ejemplo:

In [18]:
text = "Hello world?"
tokenizer.encode(text)

KeyError: 'Hello'

Por ello se requieren de sets de entrenamientos muy amplios.

## Tokens especiales de contexto

En este punto se añadirán funcioanlidades al tokenizador para que pueda trabajar con vocabulario que no tenga disponible, y además generar tokens especiales que puedan dar información contextual relevante.

Se van a añadir por tanto dos tokens, <|unk|> para palabras desconocidas por el vocabulario y <|endoftext|> para señalar que el documento/libro/texto ha finalizado y no tiene relación con el siguiente texto.

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)


('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [29]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in result if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[word] for word in preprocessed]
        return ids

    def decode (self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?_!"()\']|--)', r'\1', text)
        return text

In [32]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace"
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace


In [ ]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131]


In [34]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>


Dependiendo del tipo de LLM otros tokens recomendados son:
- [BOS]: Beginning of sequence
- [EOS]: End of sequence
- [PAD]: Padding, asegura que todos los textos tienen la misma longitud, por lo que los textos más cortos se extienden usando [PAD].

El tokenizador de GPT por ejemplo no tuiliza ninguno de estos tokens y solo utiliza endoftext.

## Byte pair encoding

Se trata de una forma mas sofisticada de tokenización que la que hemos estado usando de ejemplo, este tokenizador se ha usado para entrenar los GPT 2 y 3.

Para implementarlo se utiliza la libreria tiktoken que ya implementa el algoritmo BPE usando codigo fuente de Rust. 

Podemos instanicar el BPE tokenizer cogiendo directamente el método de codificación de gpt2.

In [35]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [38]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace"
text = " <|endoftext|> ".join((text1, text2))

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 262, 20562]


In [39]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace


El tokenizador BPE de los modelos GPT tiene un tamaño de vocabuilario de 50257, donde se asigna <|endoftext|> al valor más grande.

El algoritmo de BPE además puede trabajar con palabras desconocidas transformando estas palabras en subpalabras más pequeñas, incluso a caractéres individuales, de tal forma puede trabajar con palabras incluso desconocidas para poder trabajar con ellas.

Su vocabulario se construye uniendo caractéres frecuentes en subpalabras y subpalabras frecuentes en palabras, todo de forma iterativa.

## Data sampling

El siguiente paso es generar los pares de input-target que se requieren para entrenar un LLM.

Se implementará un cargador de datos que extrae los pares de input-target del conjunto de entrenamiento de datos utilizando un método "sliding-window".

In [40]:
with open("the-veredict.txt", 'r', encoding='utf-8') as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5146


In [42]:
enc_sample = enc_text[:50]

Una de las formas intuitivas para crear pares input-target para tareas de predicción de la siguiente palabra consiste en crear dos variables, x e y, donde x contiene los input tokens e y los targets, que serán los inputs movidos una posición.

In [47]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:     {y}")

x: [40, 367, 2885, 1464]
y:     [367, 2885, 1464, 1807]


De esta forma se pueden crear las tareas de predicción de siguiente palabra.

In [48]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[40] ----> 367
[40, 367] ----> 2885
[40, 367, 2885] ----> 1464
[40, 367, 2885, 1464] ----> 1807


In [50]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

I ---->  H
I H ----> AD
I HAD ---->  always
I HAD always ---->  thought


El último paso para convertir los tokens en embeddings es implementar un cargado de datos eficiente que itere sobre el dataset y devuelve los inputs y targets como tensores de PyTorch.

Necesitamos dos tensores, el input que es el texto que el LLM ve, y el target que incluye lo que el modelo debe predecir.

PyTorch incluye una clases Dataset y DataLoader que permiten la implementación eficiente.

In [65]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []

        tokens_ids = tokenizer.encode(txt)

        for i in range(0, len(tokens_ids) - max_length, stride):
            input_chunk = tokens_ids[i:i+max_length]
            target_chunk = tokens_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

Esta clase, basada en la clase Dataset de PyTorch, define como las filas se extraen del dataset, cada fila consiste de un númer de token IDs (basado en una longitud máxima) asignada a un tensor de input. El target tensor contiene los targets correspondientes.

El código siguiente carga los inputs en batches utilizando el DataLoader de PyTorch

In [187]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    return dataloader

In [67]:
with open("the-veredict.txt", 'r', encoding='utf-8') as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle = False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


El primer batch contiene dos tensores, el primero guarda los input token IDs, y el segundo los target. Como la longitud máxima se establece en 4 cada tensor contiene 4 IDs, en general los LLMs se entrenan con inputs de al menos 256.

El 'stride' define cuantas posiciones se van a desplazar los inputs entre batches, emulando el método de sliding window.

El tamaño del batch va a ser un hiperparámetro a ajustar, ya que tamaños menores requieren menos memoria en el entrenamiento pero dan como resultado modelos con mayor ruido.

## Embeddings

El último paso en la preparación de inputs es convertir los tokens en vectores de embedding.

Los pesos de los embeddings se inicializan con valores aleatorios como paso preliminar. Este paso sirve como punto inicial del entrenamiento del LLM, posteriormente habrá que optimizarlos.

Los embeddings (representación vectorial continua) es necesaria ya que estos modelos utilizan redes neuronales profundas con retropropagación.

Para ilustrar estos embeddings, suponemos que disponemos de un vocabulario de 6 palabras, y queremos un embedding de tamaño 3 (GPT-3 utiliza 12288 dimensiones como referencia)

In [68]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


La matriz de pesos tiene 6 filas (tamaño del vocabulario) y 3 columnas (cada dimensión del embedding).

La capa de embeddings es a efectos una operación 'look-up' que recupera filas de la matriz de pesos del embedding usando el ID del token.

## Codificación de posición

Aunque lo visto previamente es aceptable para un LLM, estos modelos no tienen una noción de la posición o el orden de los tokens en una secuencia. La capa de embeddings tiene el mismo problema.

El mecanismo de auto-atención del modelo no usa la posición, pero es útil incorporar esa información al modelo.

Existen dos categorías para embeddings de posición, los relativos y los absolutos.

Los embeddings absolutos generan para cada token un embedding posicional específico hasta el final de los tokens. Mientas que los relativos se fijan en la distancia relativa entre tokens. El modelo aprende como de lejos están los tokens.

GPT utiliza absolutos optimizados durante el entrenamiento. 

In [71]:
output_dim = 256
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInput shape:", inputs.shape)


Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Input shape: torch.Size([8, 4])


In [72]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


Para usar el método GPT de embeddings absolutos crearemos otra capa de embeddings con las mismas dimensiones que la capa de embeddings. 

In [73]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [75]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


# Mecanismos de atención

## Self atenttion with trainable weights

In [76]:
import torch
inputs = torch.tensor(
[[0.43, 0.15, 0.89], 
[0.55, 0.87, 0.66], 
[0.57, 0.85, 0.64],
[0.22, 0.58, 0.33],
[0.77, 0.25, 0.10],
[0.05, 0.80, 0.55]]
)

query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [77]:
res = 0

for idx, elemtn in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]
print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


In [78]:
attn_weights_2 = torch.softmax(attn_scores_2, dim = 0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [79]:
query = inputs[1] # 2nd input token is the query
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


Comptuing all attention weights

In [80]:
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


En lugar de usar for-loops que son muy lentos en python, se utiliza multiplicación matricial.

In [81]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [82]:
attn_weights = torch.softmax(attn_scores, dim=1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


El sumatorio de las filas dara 1 por la normalización softmax.

In [83]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [84]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## Self attention with trainable weights

En primer lugar se introducen las tres matrices entrenables de pesos, las cuales se utilizan para proyectar los tokens de input embedidos a query, llave y valor.

En primer lugar como ejemplo se va a calcular un vector contextual z(2).

In [96]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

En modelos GPT las dimensiones input y output suelen ser las mismas pero para mejor visualización se usan dimensiones distintas (3 y 2).

In [87]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False) # False para reducir clutter visual, pero para entrenamiento debería estar en True

query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([-1.1729, -0.0048])


En las matrices de pesos W, los pesos son los parámetros de pesos (valores de la nn optimizados) no los pesos de atención.

Estos pesos son los coeficientes fundamentales que definen las conexiones de la red, mientras que los atencionales son valores dinámicos dependientes del contexto.

In [88]:
keys = inputs @ W_key
values = inputs @ W_value
print(keys.shape)
print(values.shape)

torch.Size([6, 2])
torch.Size([6, 2])


A continuación se computan las puntuaciones atencionales, con un dot-product, usando la query y la llave obtenidas de transformar los inputs con sus respectivas matrices de pesos.

In [89]:
keys_2 = keys[1]
attn_scores_2 = query_2.dot(keys_2)
print(attn_scores_2)

tensor(-0.4683)


In [90]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([-0.2218, -0.4683, -0.4598, -0.2833, -0.1766, -0.3816])


A continuación pasasmos de las puntuaciones atencionals a los pesos atencionales. Usando la función softmax para normalizarlos y los escalamos utilizando la raiz cuadrada de las dimensiones del embedding de la llave.

In [91]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / (d_k ** 0.5), dim=-1)
print(attn_weights_2)


tensor([0.1796, 0.1509, 0.1518, 0.1719, 0.1854, 0.1604])


La normalización por la dimension del embedding es para mejorar el rendimiento del entrenamiento evitando pequeños gradientes. Cuando se escalan dimensiones en modelos con muchas dimensiones como los GPT, los dot-products pueden generar gradientes muy pequeños durante la retropropagación debido al escalado softmax. Esta reducción puede reducir mucho el entrenamiento.

Por úlitmo computamos los vectores contextuales como una suma ponderada sobre los vectores input.

In [92]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.9768, 1.1317])


### Implementación en una clase compacta para generalizar

La clase se deriva del nn.Module, una funcionalidad de PyTorch para crear modelos. 

El método init inicializa las matrices de pesos para query, llave y valor, cada una transformando la dimensión input a una dimensión output.

Durante el forward pass, computamos las puntuaciones de atención multiplicando queries y llaves y normalizando con softmax. Por úlitmo se crea el vector de contexto ponderando los valores con las puntuaciones normalizadas.

In [100]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.randn(d_in, d_out))
        self.W_key = nn.Parameter(torch.randn(d_in, d_out))
        self.W_value = nn.Parameter(torch.randn(d_in, d_out))
    
    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(attn_scores / (self.d_out ** 0.5), dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [99]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2845, 0.4071],
        [0.2854, 0.4081],
        [0.2854, 0.4075],
        [0.2864, 0.3974],
        [0.2863, 0.3910],
        [0.2860, 0.4039]], grad_fn=<MmBackward0>)


En lugar de utilizar nn.Prameter, se modifica la clase para que utilice nn.Linear, la cual ya tiene un patrón de inicialización de optimización de pesos, para que el entrenamiento sea más estable.

In [102]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias = qkv_bias)
    
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        context_vecs = attn_weights @ values
        return context_vecs

In [103]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


## Casual or masked attention

Este tipo de atención restringe el modelo para que solo considere inputs previos y actuales en una secuencia cuando procesa un token. Contrasta con el modelo estándar, que permite acceder a la secuencia completa.

Cuando se computan las puntuaciones atencionales, los mecanismos se aseguran que el modelo solo tiene en cuenta los tokens que ocurren durante o antes del token actual en la secuencia.

Para este fin, en modelos GPT para cada token procesado se ocultan los tokens futuros. Se enamscaran los pesos atencionales sobre la diagonal de la matriz de pesos.

Primero computamos los pesos atencionales usando softmax como en pasos previos.

In [104]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


Se implementa una mascara utilizando la función tril, donde los valores sobre la diagonal son 0.

In [105]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [107]:
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


A continuación hay que re-normalizar los pesos para que sumen 1 por filas.

In [108]:
row_sums = masked_simple.sum(dim = 1, keepdim = True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


A priori puede parecer que la información de los tokens futuros pueden influir en el token actual porque sus valores son parte del calculo softmax, sin embargo, cuando se renormalizan los pesos post masking, se está recalculando a efectos la softmax sobre un conjunto menor.

Por tanto el efecto de las posiciones se nulifica y no contribuyen a la puntuación softmax.

Para mejorar la eficiencia de la computación, podemos aprovechar las propiedades de la función softmax para hacer el proceso en menos pasos.

La función softmax convierte los inputs en una distribución de probabilidad, cuando existen valores infinitos negativos en una fila, la función softmax los trata como un valor de probabilidad 0 (porque $e^{-\inf}$ se aproxima a 0)

In [109]:
mask= torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [110]:
attn_weights = torch.softmax(masked / keys.shape[-1] ** 0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


Para reducir además el overfitting al entrenar el LLM, se incopora droput en el masking.

En la arquitectura de transformer, el dropout se aplica en dos areas, después de calcular la puntuación de atención o después de aplicar los pesos atencionales a los vectores valor.

La forma más común es aplicarlo después de computar los pesos atencionales.

In [111]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [112]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


Vamos a incoporar todo a la clase que creamos previamente

In [114]:
batch = torch.stack((inputs, inputs), dim = 0)
print(batch.shape)

torch.Size([2, 6, 3])


In [113]:
class CasualAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias = False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
    
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)   
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask[:num_tokens, :num_tokens].bool(), -torch.inf
        )
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        context_vec = attn_weights @ values
        return context_vec


In [115]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CasualAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print(context_vecs.shape)

torch.Size([2, 6, 2])


## Multi-head attention

Este término refiere a dividir el mecanismo de atención en multiples "cabezas" que operan de forma independiente. 

Una forma sencilla de implementar este tipo de modelo sería crear varias instancias del mecanismo de atención previo, cada una con sus pesos, y luego combinar sus outputs.

In [117]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_head, qvk_bias = False):
        super().__init__()
        self.heads = nn.ModuleList([
            CasualAttention(d_in, d_out, context_length, dropout, qvk_bias)
            for _ in range(num_head)
        ])
    
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [118]:
torch.manual_seed(123)
context_length = batch.shape[1] # Number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, 2)
context_vecs = mha(batch)

print(context_vecs)
print(context_vecs.shape)


tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
torch.Size([2, 6, 4])


Este método aunque funcional, es más exigente en los recursos y computacionalmente ineficiente, vamos a proceder a hacer computación en paralelo.

In [125]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qvk_bias = False):
        super().__init__()
        assert d_out % num_heads == 0 # d_out must be divisible by num_head
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias = qvk_bias)
        self.W_key = nn.Linear(d_in, d_out, bias = qvk_bias)
        self.W_value = nn.Linear(d_in, d_out, bias = qvk_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
    
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec

In [130]:
batch.shape

torch.Size([2, 6, 3])

In [134]:
torch.manual_seed(123)
batch_size, context_length, _ = batch.shape
d_in = 768
batch = torch.randn(batch_size, context_length, d_in)
d_out = 768
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=12)
context_vecs = mha(batch)
print(context_vecs)
print(context_vecs.shape)

tensor([[[-0.4122, -0.3407,  0.1825,  ..., -0.5287,  0.3060, -0.0141],
         [-0.1348, -0.0271,  0.2732,  ..., -0.3325,  0.3267, -0.0624],
         [-0.1786, -0.0997, -0.0029,  ..., -0.1884,  0.1355,  0.1337],
         [-0.1867, -0.1445, -0.0133,  ..., -0.0970,  0.1610,  0.0178],
         [-0.1981,  0.0331,  0.1722,  ..., -0.1526,  0.1174, -0.1221],
         [-0.1294,  0.0478,  0.1952,  ..., -0.0596,  0.0435,  0.1154]],

        [[-0.0932, -0.3314,  0.0504,  ...,  0.2113,  0.0064, -0.1676],
         [ 0.1388, -0.1032,  0.3344,  ...,  0.3218,  0.0954, -0.2578],
         [ 0.2299, -0.0454,  0.4388,  ...,  0.1252,  0.0237, -0.2852],
         [ 0.1257,  0.0040,  0.3551,  ...,  0.0675,  0.0847, -0.2617],
         [ 0.2107,  0.0909,  0.1148,  ..., -0.0023, -0.0152, -0.0926],
         [ 0.1475,  0.0250,  0.0836,  ..., -0.0189,  0.0284, -0.0214]]],
       grad_fn=<ViewBackward0>)
torch.Size([2, 6, 768])


# Implementacion de un modelo GPT

In [144]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "num_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qvk_bias": False
}

Clase placeholder para la arquitectura de un GPT.

In [146]:
import torch
import torch.nn as nn

class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )
    
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits
    
class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        return x
    
class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps = 1e-5):
        super().__init__()

    def forward(self, x):
        return x

In [135]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [147]:
torch.manual_seed(123)
model = DummyGPTModel(GPT_CONFIG_124M)
logits = model(batch)
print(logits.shape)
print(logits)

torch.Size([2, 4, 50257])
tensor([[[-0.9289,  0.2748, -0.7557,  ..., -1.6070,  0.2702, -0.5888],
         [-0.4476,  0.1726,  0.5354,  ..., -0.3932,  1.5285,  0.8557],
         [ 0.5680,  1.6053, -0.2155,  ...,  1.1624,  0.1380,  0.7425],
         [ 0.0447,  2.4787, -0.8843,  ...,  1.3219, -0.0864, -0.5856]],

        [[-1.5474, -0.0542, -1.0571,  ..., -1.8061, -0.4494, -0.6747],
         [-0.8422,  0.8243, -0.1098,  ..., -0.1434,  0.2079,  1.2046],
         [ 0.1355,  1.1858, -0.1453,  ...,  0.0869, -0.1590,  0.1552],
         [ 0.1666, -0.8138,  0.2307,  ...,  2.5035, -0.3055, -0.3083]]],
       grad_fn=<UnsafeViewBackward0>)


## Normalización de activaciones

In [149]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim = True)
        var = x.var(dim=-1, keepdim = True, unbiased = False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [150]:
ln = LayerNorm(5)
torch.manual_seed(123)
batch_example = torch.randn(2, 5)
out_ln = ln(batch_example)
mean = out_ln.mean(dim=-1)
var = out_ln.var(dim=-1, unbiased = False)
print(mean)
print(var)

tensor([-2.9802e-08,  0.0000e+00], grad_fn=<MeanBackward1>)
tensor([1.0000, 1.0000], grad_fn=<VarBackward0>)


## Feed forward network with GELU

In [154]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
        torch.sqrt(torch.tensor(2.0 / torch.pi)) *
        (x + 0.044715 * torch.pow(x, 3))
    ))

In [156]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )
    
    def forward(self, x):
        return self.layers(x)

In [157]:
ffn = FeedForward(GPT_CONFIG_124M)
x = torch.rand(2, 3, 768)
out = ffn(x)
print(out.shape)

torch.Size([2, 3, 768])


## Conexión de capas de atención y lineales en un transformer

In [160]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in = cfg["emb_dim"],
            d_out = cfg["emb_dim"],
            context_length = cfg["context_length"],
            num_heads = cfg["num_heads"],
            dropout=cfg["drop_rate"],
            qvk_bias = cfg["qvk_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_resid = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_resid(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_resid(x)
        x = x + shortcut
        return x

In [162]:
torch.manual_seed(123)
x = torch.randn(2, 4, 768)
block = TransformerBlock(GPT_CONFIG_124M)
out = block(x)

print(x.shape)
print(out.shape)

torch.Size([2, 4, 768])
torch.Size([2, 4, 768])


In [163]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [164]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.1381,  0.0077, -0.1963,  ..., -0.0222, -0.1060,  0.1717],
         [ 0.3865, -0.8408, -0.6564,  ..., -0.5163,  0.2369, -0.3357],
         [ 0.6989, -0.1829, -0.1631,  ...,  0.1472, -0.6504, -0.0056],
         [-0.4290,  0.1669, -0.1258,  ...,  1.1579,  0.5303, -0.5549]],

        [[ 0.1094, -0.2894, -0.1467,  ..., -0.0557,  0.2911, -0.2824],
         [ 0.0882, -0.3552, -0.3527,  ...,  1.2930,  0.0053,  0.1898],
         [ 0.6091,  0.4702, -0.4094,  ...,  0.7688,  0.3787, -0.1974],
         [-0.0612, -0.0737,  0.4751,  ...,  1.2463, -0.3834,  0.0609]]],
       grad_fn=<UnsafeViewBackward0>)


In [169]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)

        logits = logits[:, -1, :]
        probas = torch.softmax(logits, dim=-1)
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)
        idx = torch.cat((idx, idx_next), dim=-1)
    return idx

In [166]:
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [170]:
model.eval() #A
out = generate_text_simple(
    model=model,
    idx=encoded_tensor,
    max_new_tokens=6,
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output:", out)
print("Output length:", len(out[0]))


Output: tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267]])
Output length: 10


In [171]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am Featureiman Byeswickattribute argue


# Training the model

In [172]:
import tiktoken

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0)
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model = model,
    idx = text_to_token_ids(start_context, tokenizer),
    max_new_tokens = 10,
    context_size = GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you Aeiman Byeswickattributeometer inspector Normandy freezerigrate


## Calcular la pérdida de genración de texto

Se busca cuantificar la calidad del texto generado durante el entrenamiento con el valor de pérdida.

In [205]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 256, #A
    "emb_dim": 768,
    "num_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    #B
    "qvk_bias": False
}

In [194]:
file_path = "the-veredict.txt"
with open(file_path, 'r', encoding='utf-8') as f:
    text_data = f.read()

total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))
print("Characters:", total_characters)
print("Tokens:", total_tokens)

train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

Characters: 20480
Tokens: 5146


In [195]:
train_loader = create_dataloader_v1(
    train_data,
    batch_size = 2,
    max_length = GPT_CONFIG_124M["context_length"],
    stride = GPT_CONFIG_124M["context_length"],
    drop_last = True,
    shuffle=True
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size = 2,
    max_length = GPT_CONFIG_124M["context_length"],
    stride = GPT_CONFIG_124M["context_length"],
    drop_last = True,
    shuffle = False
)

In [196]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)
print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)

Train loader:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])

Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])


In [197]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(
        logits.flatten(0, 1), target_batch.flatten()
    )
    return loss

In [198]:
def calc_loss_loader(data_loader, model, device, num_batches = None):
    total_loss = 0.0
    if num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [199]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #A
model.to(device)
train_loss = calc_loss_loader(train_loader, model, device) #B
val_loss = calc_loss_loader(val_loader, model, device)
print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Training loss: 10.988501760694716
Validation loss: 10.994744300842285


In [201]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs, eval_freq, eval_iter, start_context):
    train_losses, val_losses, track_tokens_seen = [], [], [] #A
    tokens_seen, global_step = 0, -1

    for epoch in range(num_epochs):
        model.train()
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad()
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward()
            optimizer.step()
            tokens_seen += input_batch.numel()
            global_step += 1

            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter
                )
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch + 1} (Step {global_step:06d}): "
                      f"Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}")
            
        generate_and_print_sample(
            model, train_loader.dataset.tokenizer, device, start_context
        )
    return train_losses, val_losses, track_tokens_seen

def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
        decoded_text = token_ids_to_text(token_ids, tokenizer)
        print(decoded_text.replace("\n", " ")) # Compact print format
    model.train()

In [206]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.01)
num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=1,
    start_context="Every effort moves you"
)

Ep 1 (Step 000000): Train loss: 9.8299, Val loss: 9.9268
Ep 1 (Step 000005): Train loss: 8.1327, Val loss: 8.3339
Every effort moves you,,,,,,,,,,,,.                                     
Ep 2 (Step 000010): Train loss: 6.7698, Val loss: 7.0469
Ep 2 (Step 000015): Train loss: 6.4965, Val loss: 6.5731
Every effort moves you, and,, and, and,,,,, and, and,,,,,,,,,,,,,, and,,,, and,, and,,,,, and,,,,,,
Ep 3 (Step 000020): Train loss: 5.5776, Val loss: 6.4932
Ep 3 (Step 000025): Train loss: 4.7300, Val loss: 6.3858
Every effort moves you, and to the picture.                      "I, and the of the of the's the honour, and, and I had been, and I
Ep 4 (Step 000030): Train loss: 5.2804, Val loss: 6.3612
Ep 4 (Step 000035): Train loss: 3.8497, Val loss: 6.2618
Every effort moves you of the to the picture to the of the picture--and by his of the picture to have to        "I was his the picture and I had the picture.          
Ep 5 (Step 000040): Train loss: 3.6575, Val loss: 6.1981
Every effort m

Inclusión de temperatura y top-k para mejorar la generación de texto

In [219]:
def generate(model, idx, max_new_tokens, context_size, temperature = 0.0, top_k=None, eos_id = None):
    for _ in range(max_new_tokens): #A
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]
        if top_k is not None: #B
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(
                logits < min_val,
                torch.tensor(float('-inf')).to(logits.device),
                logits
            )
        if temperature > 0.0: #C
            logits = logits / temperature
            probs = torch.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
        else: #D
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)
        if idx_next == eos_id:
            break
        idx = torch.cat((idx, idx_next), dim=1)
    return idx

In [221]:
torch.manual_seed(123)
token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=15,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=25,
    temperature=1.4
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you'd escape through the and Jack! But you can: "When but I
